# Actividad 03: Reglas de Asociación
```
Docente: CARLOS FERNANDO MONTOYA CUBAS
Alumno: MANUEL HUMBERTO VELARDE FLORES
```

## Importación de Librerías
Librerías indicadas en el enunciado del trabajo

In [1]:
import numpy as np
import pandas as pd
from itertools import *
import networkx as nx

## Lectura de archivo

In [2]:
spotify = np.load('spotify.npy',allow_pickle=True,encoding="latin1")
spotify = spotify.tolist()
playlists = []
for k in spotify:
  playlists.append(spotify[k])

## Implementación del algoritmo apriori y reglas de asociación

### Implementación de métodos de apoyo

In [58]:
def conteoApariciones(lista_de_playlists):
  # Generar un diccionario para almacenar nuestra información
  conteo = {}
  # Asignar el tamaño de nuestra información a una variable
  tamanio_lista = len(lista_de_playlists)
  # Recorrer para determinar las apariciones
  for i in range(tamanio_lista):
    tamanio_playlist = len(lista_de_playlists[i])
    for j in range(tamanio_playlist):
      cancion = lista_de_playlists[i][j]
      # Aumentar un punto al conteo y guardar el número de playlist en la
      # que se encuentra
      if cancion in conteo:
        conteo[cancion][0] += 1
        conteo[cancion][1].add(i)
        pass
      # Crear un nuevo elemento
      else:
        conteo[cancion] = [1,{i}]
  return conteo

def conteoNItemsets(itemset,iteracionPrevia,longDataset):
  nuevoItemset = []
  nuevaIteracion = []
  for i in range(0,len(itemset)-1):
    for j in range(i+1,len(itemset)):
      # Combinar elementos de longitud N para ver el número de repeticiones
      combinacion = sorted(list(set(itemset[i]+itemset[j])))
      # Si la combinación solo incrementa su longitud en 1 es válida
      if(len(combinacion)==len(itemset[0])+1 and combinacion not in nuevoItemset):
      # Creamos el nuevo Itemset
        posicionesPlaylists = iteracionPrevia[i][2]&iteracionPrevia[j][2]
        nuevoItemset.append(combinacion)
        nuevaIteracion.append([combinacion,len(list(posicionesPlaylists))/longDataset,posicionesPlaylists])
  return nuevaIteracion

In [102]:
def obtenerReglas(arreglo):
  # Hacemos uso de permutaciones para considerar nuestas posibles reglas
  permutaciones = list(permutations(arreglo))
  arregloX = []
  arregloY = []
  for permutacion in permutaciones:
    permutacion = list(permutacion) # convertir en arreglo
    # Obtener las posibles separaciones considerando que en X -> Y
    # X y Y tiene una longitud mínima de 1
    for i in range(1,len(permutacion)):
      # Realizamos las separaciones
      X, Y = permutacion[:i],permutacion[i:]

      # Ordenamos nuestras separaciones para eliminar repetidos
      X.sort()
      Y.sort()

      # Las agregamos al total
      arregloX.append(X)
      arregloY.append(Y)

  # Quitamos los duplicados
  arregloXAux = []
  for i in arregloX:
    if i not in arregloXAux:
      arregloXAux.append(i)
  arregloYAux = []
  for i in arregloY:
    if i not in arregloYAux:
      arregloYAux.append(i)
  arregloX = arregloXAux
  arregloY = arregloYAux
  return arregloX,arregloY

### Implementación de métodos principales

In [60]:
def get_frequent_itemsets(playlists, min_support):
  # Definimos dos arreglos para almacenar nuestros itemsets y su soporte
  itemsets = []
  valores = []
  
  # Creamos una variable para determinar la iteración en la que nos encontramos
  iter = 1
  print("Iteración número ===> ",iter)
  # Realizamos el procedimiento para tratar la primera iteración
  # Obtenemos el itemset de 1 elemento, el cual estará en un diccionario
  diccionarioConteo = conteoApariciones(playlists)

  # Normalizamos el conteo para tratarlo como porcentajes 
  for i in diccionarioConteo:
    diccionarioConteo[i][0] = diccionarioConteo[i][0]/len(playlists)
  
  # Filtramos los elementos que superan el umbral
  # Creamos un auxiliar para poder recorrer pues el diccionario se 
  # actualiza con cada iteración
  diccionarioConteoAux = diccionarioConteo.copy() 
  for i in diccionarioConteoAux:
    if diccionarioConteo[i][0] < min_support:
      diccionarioConteo.pop(i)
  print("Cantidad de elementos filtrados: ", (len(diccionarioConteo)))

  itemsets.append(list(diccionarioConteo.keys()))
  valoresObtenidos = list(diccionarioConteo.values())
  valoresAux=[]
  for i in valoresObtenidos:
    valoresAux.append(i[0])
  valores.append(valoresAux)

  # Realizamos el procedimiento para tratar las demás iteraciones
  # Pasamos de diccionario a arreglos debido a que no se pueden tener como key
  # a más de un elemento
  # Procesamos la iteración de dos elementos para realizar el cambio
  canciones = list(diccionarioConteo.keys())
  conteoDosItemsets = []
  for i in range(len(canciones)-1):
    for j in range (i+1, len(canciones)):
      # Verificar que las canciones se encuentren en la misma playlist
      auxApariciones = diccionarioConteo[canciones[i]][1] & diccionarioConteo[canciones[j]][1]
      cantApariciones = len(list(auxApariciones))
      conteoDosItemsets.append([[canciones[i],canciones[j]],cantApariciones/len(playlists),auxApariciones])

  # Filtrar los datos de la nueva estructura
  conteoNItemsetsFiltrado = []
  for i in range(len(conteoDosItemsets)):
    if conteoDosItemsets[i][1] >= min_support:
      conteoNItemsetsFiltrado.append(conteoDosItemsets[i])
  
  # Almacenar el itemset y los valores de la nueva iteración para saber si continúa o no la poda
  itemsetActual = []
  valoresActual = []
  for i in conteoNItemsetsFiltrado:
    itemsetActual.append(i[0])
    valoresActual.append(i[1])
  
  # Creamos bandera para controlar el bucle
  seguir = True
  iter += 1
  print("Iteración número ===> ",iter)
  print("Cantidad de elementos filtrados: ", (len(conteoNItemsetsFiltrado)))
  while(True):
    # Almacenamos los itemsets y sus valores  
    itemsets.append(itemsetActual)
    valores.append(valoresActual)
    # Se termina si no se tienen más de 2 elementos
    if len(conteoNItemsetsFiltrado) <= 1:
      print("Iteraciones finalizadas")
      break
    # Determinamos que nos encontramos en una nueva iteración válida
    iter+=1
    print("Iteración número ===> ",iter)
    # Creación del nuevo itemset
    nuevoItemset = conteoNItemsets(itemsetActual, conteoNItemsetsFiltrado, len(playlists))
    
    # Realizamos la limpieza del nuevo itemset
    conteoNItemsetsFiltrado = []
    for i in range(len(nuevoItemset)):
      if nuevoItemset[i][1] >= min_support:
        conteoNItemsetsFiltrado.append(nuevoItemset[i])

    # Actualizamos para la nueva iteracion
    itemsetActual = []
    valoresActual = []
    for i in conteoNItemsetsFiltrado:
      itemsetActual.append(i[0])
      valoresActual.append(i[1])

    print("Cantidad de elementos filtrados: ", (len(conteoNItemsetsFiltrado)))
  return itemsets, valores

In [148]:
def generate_association_rules(frequent_itemsets, confidence = 0, lift = 0):
  arregloReglas = []
  arregloLifts = []
  arregloConfianzas = []
  arregloSoportes = []
  itemsets = frequent_itemsets[0]
  valores = frequent_itemsets[1]
  arregloX = []
  arregloY = []
  longRegla = 2
  # Obtención de posibles reglas
  while longRegla <= len(itemsets):
    itemsetAux = itemsets[longRegla-1]
    for elemento in itemsetAux:
      X, Y = obtenerReglas(elemento)
      arregloX += X
      arregloY += Y
    longRegla += 1
  # Obtención de lift Y confianza
  for i in range(len(arregloX)):
    X = arregloX[i]
    Y = arregloY[i]
    XuY = X+Y
    # Obtención de valores en base a la teoría
    # Para X
    posicionElemento = itemsets[len(X)-1].index(X[0])
    Px = valores[len(X)-1][posicionElemento]
    # Para Y
    posicionElemento = itemsets[len(Y)-1].index(Y[0])
    Py = valores[len(Y)-1][posicionElemento]
    # Para X U Y
    try:
      posicionElemento = itemsets[len(XuY)-1].index(XuY)
      PXuY = valores[len(XuY)-1][posicionElemento]
    except:
      a=0
    # Cálculo de confianza
    confianza = PXuY/Px
    # Cálculo de lift
    liftActual = PXuY/Px/Py
    # Validación con el umbral
    if(liftActual >= lift and confianza >= confidence):
      arregloReglas.append([X,Y])
      arregloConfianzas.append(confianza)
      arregloLifts.append(liftActual)
      arregloSoportes.append(PXuY)
  return  arregloReglas,arregloConfianzas,arregloLifts,arregloSoportes

## Uso del algoritmo apriori con el dataset spotify

### Obtención de itemsetsfrecuentes
- Se puede observar la cantidad de elementos hallados en cada iteración usando el umbral de 0.015

In [203]:
itemsets = get_frequent_itemsets(playlists, 0.014)

Iteración número ===>  1
Cantidad de elementos filtrados:  434
Iteración número ===>  2
Cantidad de elementos filtrados:  28
Iteración número ===>  3
Cantidad de elementos filtrados:  0
Iteraciones finalizadas


- Los datos obtenidos en los itemsets son:

In [165]:
print("ITEMSETS")
print(itemsets[0])
print("VALORES")
print(itemsets[1])

ITEMSETS
[['Crazy In Love', "It Wasn't Me", 'Yeah!', 'Say My Name', 'Hey Ya! - Radio Mix / Club Mix', 'Promiscuous', 'Party In The U.S.A.', 'Forever', 'Closer', 'Mr. Brightside', 'All The Small Things', 'Home', 'Billie Jean', 'Seven Nation Army', 'Ignition - Remix', "Jessie's Girl", "Don't Stop Believin'", 'Bohemian Rhapsody - Remastered 2011', 'No Role Modelz', 'Stronger', 'No Type', 'Lean On (feat. MØ & DJ Snake)', 'Summertime', 'Trouble', 'All I Want', 'Do I Wanna Know?', 'Chasing Cars', 'Riptide', 'How to Save a Life', 'Weak', 'Runaway', 'Intro', 'Mercy', 'What They Want', 'Issues', 'Gravity', 'One Dance', 'Sober', 'All Time Low', 'Your Song', 'Wonderwall - Remastered', 'Springsteen', 'Santeria', 'Brown Eyed Girl', 'Redbone', 'Body Like A Back Road', 'My Girl', 'I Want You Back', 'Starving', 'Smile', 'Crazy', 'Sweet Home Alabama', "Sweet Child O' Mine", 'Despacito - Remix', "I'm the One", 'Hello', 'Jungle', "'Till I Collapse", 'Ghost', 'Uptown Funk', 'V. 3005', 'Where Are Ü Now (wi

- Generar reglas de asociación

In [218]:
datosReglas = generate_association_rules(itemsets,confidence=0.5,lift=1.4)
reglasObtenidas = datosReglas[0]
confianzasObtenidas = datosReglas[1]
liftsObtenidos = datosReglas[2]
soportesObtenidos = datosReglas[3]

- Mostrar reglas de asociación con valores de confianza, lift y support obtenidos

In [235]:
for i in range(len(reglasObtenidas)):
  print("==================== Regla: ",i+1," ===========================")
  print("|"+reglasObtenidas[i][0][0], "---------->", reglasObtenidas[i][1][0])
  print("|Confianza:\t\t", confianzasObtenidas[i])
  print("|Lift:\t\t\t", liftsObtenidos[i])
  print("|Soporte:\t\t", soportesObtenidos[i])

==================== Regla:  1  ===========================
|Cold Water (feat. Justin Bieber & MØ) ----------> Closer
|Confianza:		 0.5032467532467533
|Lift:			 7.906172812580085
|Soporte:		 0.0145
==================== Regla:  2  ===========================
|Bounce Back ----------> HUMBLE.
|Confianza:		 0.5126582278481012
|Lift:			 11.063829787234043
|Soporte:		 0.0156
==================== Regla:  3  ===========================
|Bounce Back ----------> Bad and Boujee (feat. Lil Uzi Vert)
|Confianza:		 0.5158227848101264
|Lift:			 15.958451369216244
|Soporte:		 0.0169
==================== Regla:  4  ===========================
|goosebumps ----------> Congratulations
|Confianza:		 0.52
|Lift:			 12.27431105479886
|Soporte:		 0.0155
==================== Regla:  5  ===========================
|goosebumps ----------> HUMBLE.
|Confianza:		 0.521951219512195
|Lift:			 11.53633600442111
|Soporte:		 0.0167
==================== Regla:  6  ===========================
|XO TOUR Llif3 ----------> Co

## Explicación de reglas obtenidas
- Seleccionando las 4 reglas con mayor confianza



1.   DNA. ----------> HUMBLE.
- Confianza:		 0.8189655172413793
- Lift:			 17.424798239178283
- Soporte:		 0.019
2.   Slippery (feat. Gucci Mane) ----------> HUMBLE.
- Confianza:		 0.6455696202531646
- Lift:			 13.06612518760484
- Soporte:		 0.0148
3.   Mask Off ----------> HUMBLE.
- Confianza:		 0.6181818181818182
- Lift:			 13.735523835173714
- Soporte:		 0.0204
4.   Mask Off ----------> XO TOUR Llif3
- Confianza:		 0.6141078838174274
- Lift:			 15.63099347909474
- Soporte:		 0.0163

```
Se puede ver que pese a que las confianzas son las más altas, los lifts no son precisamente los más altos, pero se mantienen cerca al top.
Pero los soportes no son tan altos en comparación a los de otras reglas.
```


### Considerando las canciones:
1. DNA

  Artista: Kendrick Lamar
2. HUMBLE

  Artista: Kendrick Lamar
3. Slippery (feat. Gucci Mane)

  Artista: Migos
4. Mask Off

  Artista: Future
5. XO TOUR Llif3

  Artista: Lil Uzi Vert




- Podemos ver que el Artista Kendrick Lamar tiene dos temas destacables DNA y HUMBLE, siendo una sola regla la que contiene a ambos, además de que HUMBLE aparece en dos reglas adicionales, otra información a destacar es que estos dos temas son del mismo disco. Lo cual nos puede hacer notar que fue un momento destacable para el artista.
- Algo a considerar es que todos los temas son considerados dentro del ámbito del Hip Hop, tanto en los géneros de rap y trap, y que, al mismo tiempo, todos estos artistas son de raza afroamericana.
- Todos los artistas de la lista son varones, lo cual nos puede hacer notar que en este ámbito las mujeres no alcanzaron una popularidad muy alta, al menos en estos géneros musicales, pese a haber artitas conocidas como Iggy Azalea, Nicki Minaj o Cardi B.
- El punto anterior nos hace notar que algo importante es conocer la fecha de estas canciones, por lo cual, tras buscar esta información, se encontró que todas las canciones salieron en el año 2017, año en el que artistas femeninas no tuvieron tanto realce.